In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [69.5 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://secu

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-10-17 16:13:26--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.22MB/s    in 0.8s    

2021-10-17 16:13:28 (1.22 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_01.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   22480053|R28HBXXO1UEVJT|0843952016|      34858117|          The Rising|           Books|          5|            0|          0|   N|                N|Great Twist on Zo...|I've known about ...| 2012-05-03|
|         US|   44244451| RZKRFS2UUMFFU|031088926X|     676347131|Sticky Faith Teen...|           Books|          5|    

In [5]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R28HBXXO1UEVJT|          5|            0|          0|   N|                N|
| RZKRFS2UUMFFU|          5|           15|         15|   N|                Y|
|R2WAU9MD9K6JQA|          3|            6|          8|   N|                N|
|R36SCTKYTVPZPC|          5|           10|         11|   N|                Y|
|R10BM6JUOJX27Q|          3|            0|          0|   Y|                N|
| RCLZ5OKZNUSY4|          5|            0|          0|   N|                Y|
|R1S65DJYEI89G4|          4|            8|         17|   N|                N|
|R3KQYBQOLYDETV|          4|            2|          2|   N|                N|
|R3QV8K7CSU8K2W|          5|            0|          0|   N|                N|
|R3W5A1WUGO5VQ0|          4|            0|          1|   N|     

In [6]:
# Create the total_votes DataFrame
# Filter the data and create a new DataFrame to retrieve all the rows where the total_votes count is equal to or greater than 20
total_votes_df = vine_df.filter("total_votes >= 20")
total_votes_df.show(truncate=False)

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R18VIM840CEFRP|1          |16           |105        |N   |N                |
|R14PMAJTY0EAAT|4          |135          |142        |N   |Y                |
|R1363VA3TPNLVB|5          |370          |388        |N   |Y                |
|RBQZC5A3TSWT5 |5          |11           |22         |N   |Y                |
|RW00TDPV9U93E |1          |35           |76         |N   |N                |
|R3OW0AIVLEDIQ7|3          |99           |121        |N   |N                |
|R3DTESO4FUAPKQ|5          |90           |102        |N   |N                |
|R18I0XBYWFSR5T|1          |16           |50         |N   |N                |
|R1VSQ3QK30VDI0|5          |23           |35         |N   |N                |
|R3NEJBF4FXMVUU|5          |87           |90         |N   |Y    

In [7]:
# Filter helpful_votes_df, create new DataFrame to retrieve all the rows where # of helpful_votes / by total_votes is >= 50%
helpful_votes_df = total_votes_df.filter((total_votes_df.helpful_votes/total_votes_df.total_votes) >= 0.5)
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R14PMAJTY0EAAT|          4|          135|        142|   N|                Y|
|R1363VA3TPNLVB|          5|          370|        388|   N|                Y|
| RBQZC5A3TSWT5|          5|           11|         22|   N|                Y|
|R3OW0AIVLEDIQ7|          3|           99|        121|   N|                N|
|R3DTESO4FUAPKQ|          5|           90|        102|   N|                N|
|R1VSQ3QK30VDI0|          5|           23|         35|   N|                N|
|R3NEJBF4FXMVUU|          5|           87|         90|   N|                Y|
|R34ZTQTHWRHT5R|          5|           18|         20|   N|                Y|
|R367X555006ROC|          1|           39|         44|   N|                N|
| R4NCMAZCRAXHT|          3|           19|         21|   N|     

In [8]:
#Filter helpful_votes_df, create new DataFrame that retrieves rows where a review was written as part of the Vine program 
vine_part_df = helpful_votes_df.filter("vine == 'Y'")
vine_part_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2Z8083V8ZFQVZ|          3|           82|         87|   Y|                N|
| RK25TAO4GGS5G|          3|           22|         24|   Y|                N|
|R2SJQG3C6KY0M8|          3|           18|         20|   Y|                N|
|R12CHC0CB2WASU|          5|           43|         48|   Y|                N|
|R2RY328TIDXMTE|          2|           50|         61|   Y|                N|
|R3K8OQU4PEP8W7|          4|           40|         46|   Y|                N|
|R1JI1A7JT99538|          3|           28|         40|   Y|                N|
|R247K3K5SVPJUL|          3|           15|         20|   Y|                N|
|R27UEUDUYPLXZC|          5|           20|         24|   Y|                N|
|R2QQDNBYDIEL9F|          5|           35|         41|   Y|     

In [9]:
#Filter helpful_votes_df, create new DataFrame that retrieves rows where a review was written as NOT part of the Vine program 
nonvine_df = helpful_votes_df.filter("vine == 'N'")
nonvine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R14PMAJTY0EAAT|          4|          135|        142|   N|                Y|
|R1363VA3TPNLVB|          5|          370|        388|   N|                Y|
| RBQZC5A3TSWT5|          5|           11|         22|   N|                Y|
|R3OW0AIVLEDIQ7|          3|           99|        121|   N|                N|
|R3DTESO4FUAPKQ|          5|           90|        102|   N|                N|
|R1VSQ3QK30VDI0|          5|           23|         35|   N|                N|
|R3NEJBF4FXMVUU|          5|           87|         90|   N|                Y|
|R34ZTQTHWRHT5R|          5|           18|         20|   N|                Y|
|R367X555006ROC|          1|           39|         44|   N|                N|
| R4NCMAZCRAXHT|          3|           19|         21|   N|     

In [10]:
#Determine the total number of reviews
helpful_votes_df.count()

337176

In [11]:
#Determine the total number of 5 star reviews
helpful_votes_df.filter("star_rating==5").count()

170404

In [12]:
#Determine the number of 5-star reviews, first count paid reviews
paid_reviews_count = vine_part_df.count()
paid_reviews_count

4781

In [13]:
#Determine the number of 5-star reviews,  second count unpaid reviews
unpaid_reviews_count = nonvine_df.count()
unpaid_reviews_count

332395

In [14]:
#Determine the number of 5-star reviews, third filter out five star reviews from paid reviews
paid_five_star_reviews = vine_part_df.filter(vine_part_df.star_rating==5).count()
paid_five_star_reviews

1604

In [15]:
#Determine the number of 5-star reviews, fourth filter out five star reviews from UNpaid reviews
UNpaid_five_star_reviews = nonvine_df.filter(nonvine_df.star_rating==5).count()
UNpaid_five_star_reviews

168800

In [20]:
#Determine the percentage of 5-star reviews for the paid reviews, divid total paid by 5-star paid reviews
percent_paid = (paid_five_star_reviews/paid_reviews_count)*100
percent_paid

33.5494666387785

In [21]:
#Determine the percentage of 5-star reviews for the UNpaid reviews, divide total unpaid by 5-star unpaid reviews
percent_unpaid = (UNpaid_five_star_reviews/unpaid_reviews_count)*100
percent_unpaid

50.782954015553784